In [1]:
# pip install pybaseball pandas numpy pyarrow
from datetime import date
import pandas as pd
import numpy as np
from pybaseball import statcast, playerid_reverse_lookup

# ---------------------------------------------
# 1) CONFIG
# ---------------------------------------------
START = "2025-03-01"   # buffer before opening day
END   = "2025-11-20"   # buffer past WS
TEAMS = ["LAD", "TOR"] # Dodgers, Blue Jays
OUTFILE_ALL = "pitch_events_2025_LAD_TOR.csv"   # combined
OUTFILE_PER_TEAM_FMT = "pitch_events_2025_{team}.csv"

# ---------------------------------------------
# 2) DOWNLOAD ALL 2025 PITCHES (REG + POST)
# ---------------------------------------------
print("Downloading Statcast 2025 (this can take a while)...")
df = statcast(start_dt=START, end_dt=END)
df = df.copy()

# Keep only pitches (some rows may be non-pitch events)
df = df[df["pitch_type"].notna()].copy()

# Keep Regular + Postseason only if column available
if "game_type" in df.columns:
    df = df[df["game_type"].isin(["R", "P"])].copy()

# ---------------------------------------------
# 3) LIMIT TO GAMES WITH LAD OR TOR
# ---------------------------------------------
df = df[(df["home_team"].isin(TEAMS)) | (df["away_team"].isin(TEAMS))].copy()

# ---------------------------------------------
# 4) DERIVE 'pitcher_team' AND HOME/AWAY FOR PITCHER
#    Pitcher is fielding team: Top -> home fields; Bottom -> away fields
# ---------------------------------------------
def pitcher_team(row):
    return row["home_team"] if row["inning_topbot"] == "Top" else row["away_team"]

df["pitcher_team"] = df.apply(pitcher_team, axis=1)
df["home_away"] = (df["pitcher_team"] == df["home_team"]).astype(int)

# ---------------------------------------------
# 5) BASIC IDENTIFIERS / ORDERING
# ---------------------------------------------
# Statcast columns:
# - game_pk: unique game id
# - at_bat_number: PA sequence within game
# - pitch_number: order within at-bat (we'll preserve as ab_pitch_number)
# We'll also build a GLOBAL pitch number per game (all pitches, any pitcher/team).

df = df.sort_values(["game_pk", "at_bat_number", "pitch_number"]).copy()
df["ab_pitch_number"] = df["pitch_number"].astype(int)

# Global pitch number per game (1..N across entire game)
df["pitch_number"] = (
    df.groupby("game_pk").cumcount() + 1
)

# ---------------------------------------------
# 6) NAMES FOR PITCHER & BATTER (map from MLBAM IDs)
#    pybaseball provides IDs -> names reverse lookup.
# ---------------------------------------------
# Collect unique IDs
pitcher_ids = df["pitcher"].dropna().astype(int).unique().tolist()
batter_ids  = df["batter"].dropna().astype(int).unique().tolist()
ids = sorted(set(pitcher_ids + batter_ids))

print("Resolving player names...")
name_map = playerid_reverse_lookup(ids, key_type="mlbam")[["key_mlbam", "name_first", "name_last"]]
name_map["full_name"] = name_map["name_first"].fillna("") + " " + name_map["name_last"].fillna("")
name_map = name_map.rename(columns={"key_mlbam": "id"})

# Build id -> name dict
id2name = dict(zip(name_map["id"], name_map["full_name"]))

df["pitcher_id"] = df["pitcher"].astype("Int64")
df["batter_id"]  = df["batter"].astype("Int64")
df["pitcher_name"] = df["pitcher_id"].map(id2name).fillna("")
df["batter_name"]  = df["batter_id"].map(id2name).fillna("")

# ---------------------------------------------
# 7) MATCHUP HANDS & SAME-HAND
# ---------------------------------------------
# Statcast: p_throws (R/L), stand (R/L)
df["p_hand"] = df["p_throws"]
df["b_hand"] = df["stand"]
df["same_hand_matchup"] = (df["p_hand"] == df["b_hand"]).astype(int)

# ---------------------------------------------
# 8) COUNT / INNING / TOP-BOT / RUNNERS
# ---------------------------------------------
df["balls"]   = df["balls"].astype(int)
df["strikes"] = df["strikes"].astype(int)
df["outs"]    = df["outs_when_up"].astype(int)
df["inning"]  = df["inning"].astype(int)
df["top_bot"] = (df["inning_topbot"] == "Top").astype(int)

# count_code (e.g., 3-2 -> 32 as integer)
df["count_code"] = df["balls"] * 10 + df["strikes"]

df["ahead_in_count"] = (df["strikes"] > df["balls"]).astype(int)
df["is_two_strikes"] = (df["strikes"] == 2).astype(int)
df["is_full_count"]  = (df["count_code"] == 32).astype(int)

# Runners on base: Statcast columns on_1b/on_2b/on_3b are player ids or NaN
for b in ["on_1b", "on_2b", "on_3b"]:
    if b in df.columns:
        df[b] = df[b].notna().astype(int)
    else:
        df[b] = 0  # fallback if column absent (should exist)

df["num_runners_onb"] = df[["on_1b", "on_2b", "on_3b"]].sum(axis=1)

# ---------------------------------------------
# 9) PITCHER PITCH COUNT (within game) & TIMES THROUGH ORDER
# ---------------------------------------------
# p_pitch_count: cumulative pitches thrown by that pitcher in that game
df["p_pitch_count"] = df.groupby(["game_pk", "pitcher"])["pitch_type"].cumcount() + 1

# times_through_order: based on number of DISTINCT plate appearances the pitcher has started.
# Strategy:
#   - Mark first pitch of each at-bat: ab_pitch_number == 1
#   - For each (game, pitcher), keep cumulative count of at-bats they've started while fielding
#   - TTO = floor((batters_faced_so_far - 1) / 9) + 1
is_ab_start = df["ab_pitch_number"] == 1

bf = (
    df[is_ab_start & (df["pitcher_team"].notna())]
    .groupby(["game_pk", "pitcher"], as_index=False)
    .apply(lambda g: g.assign(
        batters_faced=np.arange(1, len(g) + 1)
    ))
    .reset_index(drop=True)
    .loc[:, ["game_pk", "pitcher", "at_bat_number", "batters_faced"]]
)

# Merge BF back, forward-fill within each at-bat
df = df.merge(bf, on=["game_pk", "pitcher", "at_bat_number"], how="left")
df["batters_faced"] = df.groupby(["game_pk", "pitcher"])["batters_faced"].ffill()

# Compute TTO; if missing (reliever entered mid-inning before first AB start recorded), fill 1
df["times_through_order"] = ((((df["batters_faced"].fillna(1) - 1) // 9) + 1).astype(int))

# ---------------------------------------------
# 10) SCORE DIFF (pitcher team - opponent) BEFORE PITCH
# ---------------------------------------------
def score_diff_before_pitch(row):
    # Use current scoreboard columns from Statcast
    h, a = int(row["home_score"]), int(row["away_score"])
    if row["pitcher_team"] == row["home_team"]:
        return h - a
    else:
        return a - h

df["score_diff"] = df.apply(score_diff_before_pitch, axis=1)

# ---------------------------------------------
# 11) PREVIOUS PITCH CONTEXT (within the SAME at-bat)
# ---------------------------------------------
df = df.sort_values(["game_pk", "at_bat_number", "ab_pitch_number"]).copy()

df["prev_pitch_type"] = (
    df.groupby(["game_pk", "at_bat_number"])["pitch_type"].shift(1)
)

# 'description' is Statcast's textual result (called_strike, foul, ball, hit_into_play, etc.)
def simplify_desc(x):
    if pd.isna(x):
        return np.nan
    x = str(x)
    if "ball" in x and "blocked" not in x and "in_play" not in x:
        return "ball"
    if "called_strike" in x:
        return "called_strike"
    if "swinging_strike" in x:
        return "swinging_strike"
    if "foul" in x:
        return "foul"
    if "in_play" in x:
        return "in_play"
    if "blocked_ball" in x:
        return "ball"
    return x

df["prev_pitch_result"] = (
    df.groupby(["game_pk", "at_bat_number"])["description"]
      .shift(1)
      .map(simplify_desc)
)

# ---------------------------------------------
# 12) FINAL SELECTION & ORDER
# ---------------------------------------------
cols = [
    # identifiers / ordering
    "game_pk",              # game_id
    "at_bat_number",
    "pitch_number",         # global pitch index (per game)
    # participants
    "pitcher_name", "pitcher_id",
    "batter_name",  "batter_id",
    # matchup hands
    "p_hand", "b_hand",
    "home_away", "same_hand_matchup",
    # count / inning / runners
    "balls", "strikes", "outs", "inning", "top_bot",
    "count_code", "ahead_in_count", "is_two_strikes", "is_full_count",
    "on_1b", "on_2b", "on_3b", "num_runners_onb",
    # pitcher state
    "p_pitch_count", "times_through_order",
    # score
    "score_diff",
    # prev context
    "prev_pitch_type", "prev_pitch_result",
    # target
    "pitch_type",
    # helpful audit fields (optional but handy)
    "ab_pitch_number", "pitcher_team", "home_team", "away_team", "inning_topbot", "description"
]

missing = [c for c in cols if c not in df.columns]
if missing:
    print("WARNING: Missing columns:", missing)

out = df[ [c for c in cols if c in df.columns] ].copy()
out = out.rename(columns={"game_pk": "game_id"})

# ---------------------------------------------
# 13) WRITE FILES
# ---------------------------------------------
out.to_csv(OUTFILE_ALL, index=False)
print(f"Wrote combined: {OUTFILE_ALL} (rows: {len(out):,})")

for team in TEAMS:
    tdf = out[out["pitcher_team"] == team].copy()
    tfile = OUTFILE_PER_TEAM_FMT.format(team=team)
    tdf.to_csv(tfile, index=False)
    print(f"Wrote team file: {tfile} (rows: {len(tdf):,})")


This is a large query, it may take a moment to complete


C:\Users\rigog\anaconda3\Lib\site-packages\pybaseball\statcast.py:50: UserWarning: 
That's a nice request you got there. It'd be a shame if something were to happen to it.
We strongly recommend that you enable caching before running this. It's as simple as `pybaseball.cache.enable()`.
Since the Statcast requests can take a *really* long time to run, if something were to happen, like: a disconnect;
gremlins; computer repair by associates of Rudy Giuliani; electromagnetic interference from metal trash cans; etc.;
you could lose a lot of progress. Enabling caching will allow you to immediately recover all the successful
subqueries if that happens.
  warnings.warn(_OVERSIZE_WARNING)


Skipping offseason dates
Skipping offseason dates


C:\Users\rigog\anaconda3\Lib\site-packages\pybaseball\datahelpers\postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_datetime, errors='ignore', format=date_format)
C:\Users\rigog\anaconda3\Lib\site-packages\pybaseball\datahelpers\postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_datetime, errors='ignore', format=date_format)
C:\Users\rigog\anaconda3\Lib\site-packages\pybaseball\datahelpers\postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_

Resolving player names...
Gathering player lookup table. This may take a moment.


C:\Users\rigog\AppData\Local\Temp\ipykernel_14172\47499601.py:133: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.assign(


Wrote combined: pitch_events_2025_LAD_TOR.csv (rows: 94,466)
Wrote team file: pitch_events_2025_LAD.csv (rows: 23,959)
Wrote team file: pitch_events_2025_TOR.csv (rows: 23,711)
